In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# splinter browser setup
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/tylerhill/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


## Part  1: Scraping

### NASA Mars News

In [3]:
## designate URL for browser to visit
url = "https://redplanetscience.com"
## browser visit
browser.visit(url)

In [4]:
# Run BeautifulSoup to gather HTML for parsing
soup = bs(browser.html, 'html.parser')
# preview HTML
soup

<html><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
<link href="css/font.css" rel="stylesheet" type="text/css"/>
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
<title>News - Mars Exploration Program</title>
</head>
<body>
<div class="col-md-12">
<div class="row">
<nav class="navbar navbar-expand-lg navbar-light fixed-top">
<div class="container-fluid">
<a class="navbar-brand" href="#">
<img src="image/nasa.png" width="80"/><span class="logo">MARS Planet Science</span>
<span class="logo1">Exploration Program</spa

In [5]:
# narrow down results to find article title and description #
results = soup.find_all('div', id='news')
for result in results:
    news_title = soup.find('div', class_='content_title').text
    news_p = soup.find('div', class_='article_teaser_body').text
    print(news_title)
    print('-'*50)
    print(news_p)

Screening Soon: 'The Pathfinders' Trains Lens on Mars
--------------------------------------------------
With the Mars 2020 mission ramping up, the documentary — the first of four about past JPL missions to the Red Planet to be shown at Caltech — tells a gripping backstory.


### JPL Mars Space Images—Featured Image

In [6]:
# Grab image URL for featured Mars image
url2 = "https://spaceimages-mars.com"
browser.visit(url2)

# Grab HTML with BS
soup2 = bs(browser.html, 'html.parser')
soup2

<html class=""><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:15px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-toggle="collapse" type="button

In [7]:
# narrow down results to get desired URL
results2 = soup2.find('div', class_='floating_text_area')
link = results2.find('a', class_='showimg fancybox-thumbs')['href']
featured_image_url = f'https://spaceimages-mars.com/{link}'
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


### Mars Facts

In [8]:
# Using Pandas read_html to pull tables from galaxyfacts-mars.com
mars_fact_url = 'https://galaxyfacts-mars.com'
mars_facts = pd.read_html(mars_fact_url)
mf = pd.DataFrame(mars_facts[1])
mf

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


### Mars Hemispheres

In [73]:
# creating lists to store hemisphere names and images (incl. thumbnail)
titles = []
thumbnails = []
img_urls = []

# Hemisphere function to grab relevant info and create hemisphere dict #
# Output will be a list containing one dictionary for each hemisphere. #
# Dictionaries contain hemisphere name, image URL, thumbnail image     #

def hemispheres():
    
    # Grabbing high-res images and names for Mars hemispheres
    marhem_url = "https://marshemispheres.com/"
    browser.visit(marhem_url)
    
    # links list so that splinter has something to click on
    links = []

    # BeautifulSoup parsing, locating relevant info
    hemsoup = bs(browser.html, 'html.parser')
    hems = hemsoup.find_all('div', class_='item')

    
    # loop through all four hemispheres to grab their names and thumbnail images
    for hem in hems:
        # grab hemi title
        title = hem.h3.text
        
        # clickable link info appended to list
        links.append(title)
        
        # formatting to remove 'Enhanced', appending to list as title
        titles.append(title.replace(' Enhanced', ''))
        
        # locate thumbnail image for later use, append to list
        thumb = hem.img['src']
        thumb_url = f'https://marshemispheres.com/{thumb}'
        thumbnails.append(thumb_url)
    
    
    # loop through links list, clicking links with splinter to grab image URL
    for name in links:
        # browser clicking and set up
        browser.links.find_by_partial_text(name).click()
        soup = bs(browser.html, 'html.parser')
        
        # finding relevant link information
        hemi = soup.find('div', class_='downloads').find_all('li')
        link = hemi[0].a['href']
        
        # formatting as a complete URL, appending to list
        img_url = f'https://marshemispheres.com/{link}'
        img_urls.append(img_url)
        
        # return browser to index to click next link
        browser.visit(marhem_url)

# Run hemispheres function to create hemisphere lists
hemispheres()

In [74]:
# list comp to create hemisphere dictionary for each hemisphere

hemisphere_data = [{'title': titles[n],'img_url': img_urls[n],'thumb': thumbnails[n]} for n in range(len(titles))]
hemisphere_data

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/full.jpg',
  'thumb': 'https://marshemispheres.com/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'thumb': 'https://marshemispheres.com/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'thumb': 'https://marshemispheres.com/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'thumb': 'https://marshemispheres.com/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png'}]